In [23]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import nltk
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
import datasets
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
df = pd.read_csv('/content/drive/MyDrive/Grammar_autocorrection_df1_new.csv')

In [26]:
df.head()

,Error Type,Ungrammatical Statement,Standard English,Ungrammatical Length,Standard Length,Tokenized Ungrammatical,Tokenized Standard,POS_Ungrammatical,POS_Standard,POS_W2V_Ungram,...,W2V_Ungram,W2V_Standard,Bigrams_Ungrammatical,Bigrams_Standard,Trigrams_Ungrammatical,Trigrams_Standard,W2V_Bigrams_Ungrammatical,W2V_Bigrams_Standard,W2V_Trigrams_Ungrammatical,W2V_Trigrams_Standard
0,verb tense errors,i goes to the store everyday,i go to the store everyday,29,27,"['i', 'goes', 'to', 'the', 'store', 'everyday']","['i', 'go', 'to', 'the', 'store', 'everyday']","[('i', 'NN'), ('goes', 'VBZ'), ('to', 'TO'), (...","[('i', 'NN'), ('go', 'VBP'), ('to', 'TO'), ('t...",[ 0.06899698 -0.11690328 0.17168738 0.160985...,...,[-0.177742 0.5408214 0.27710912 0.102996...,[-0.18476208 0.565066 0.28604895 0.110692...,"['i goes', 'goes to', 'to the', 'the store', '...","['i go', 'go to', 'to the', 'the store', 'stor...","['i goes to', 'goes to the', 'to the store', '...","['i go to', 'go to the', 'to the store', 'the ...",[ 5.3249515e-04 1.2377683e-02 2.6096594e-02 ...,[ 0.00782952 0.01257214 0.03556162 0.002165...,[ 0.01734993 -0.00260671 -0.00072663 -0.003998...,[ 0.01177527 -0.00479391 -0.00123065 0.005437...
1,verb tense errors,they was playing soccer last night,they were playing soccer last night,35,36,"['they', 'was', 'playing', 'soccer', 'last', '...","['they', 'were', 'playing', 'soccer', 'last', ...","[('they', 'PRP'), ('was', 'VBD'), ('playing', ...","[('they', 'PRP'), ('were', 'VBD'), ('playing',...",[ 0.08898706 -0.16277023 0.00065081 0.091216...,...,[-0.22198214 0.6069789 0.3287705 0.110492...,[-0.20759511 0.6033656 0.326151 0.119404...,"['they was', 'was playing', 'playing soccer', ...","['they were', 'were playing', 'playing soccer'...","['they was playing', 'was playing soccer', 'pl...","['they were playing', 'were playing soccer', '...",[-0.001242 0.00464096 0.01187895 0.001658...,[ 0.00568041 0.00796309 0.00476048 0.003099...,[-2.4676307e-03 -8.5125500e-03 7.1918243e-04 ...,[ 2.0901330e-03 -5.0771208e-03 -3.5517749e-03 ...
2,verb tense errors,she have completed her homework,she has completed her homework,32,31,"['she', 'have', 'completed', 'her', 'homework']","['she', 'has', 'completed', 'her', 'homework']","[('she', 'PRP'), ('have', 'VBP'), ('completed'...","[('she', 'PRP'), ('has', 'VBZ'), ('completed',...",[-0.05448146 -0.16768712 -0.05421133 0.071171...,...,[-0.18468447 0.556527 0.2670376 0.099153...,[-0.19615619 0.5517267 0.27528045 0.090484...,"['she have', 'have completed', 'completed her'...","['she has', 'has completed', 'completed her', ...","['she have completed', 'have completed her', '...","['she has completed', 'has completed her', 'co...",[ 5.72577212e-03 1.65572483e-03 8.72172974e-...,[ 1.1315383e-03 -5.4916982e-03 1.3190885e-02 ...,[ 0.00492086 -0.00563675 -0.00285741 0.004414...,[-0.01038283 -0.00692296 -0.00054124 0.011133...
3,verb tense errors,he do not know the answer,he does not know the answer,25,27,"['he', 'do', 'not', 'know', 'the', 'answer']","['he', 'does', 'not', 'know', 'the', 'answer']","[('he', 'PRP'), ('do', 'VBZ'), ('not', 'RB'), ...","[('he', 'PRP'), ('does', 'VBZ'), ('not', 'RB')...",[-5.4591924e-02 -1.1197346e-01 -5.1746000e-02 ...,...,[-0.22064425 0.65218335 0.31709093 0.115940...,[-0.20784527 0.6110135 0.2986785 0.110379...,"['he do', 'do not', 'not know', 'know the', 't...","['he does', 'does not', 'not know', 'know the'...","['he do not', 'do not know', 'not know the', '...","['he does not', 'does not know', 'not know the...",[-0.00033541 0.00086477 0.01327611 -0.009891...,[-0.00465144 -0.00187735 0.00977874 -0.007753...,[ 1.1856072e-03 -2.4600534e-03 -2.7238706e-04 ...,[ 4.04477399e-03 -9.81178135e-03 5.62989060e-...
4,verb tense errors,the sun rise in the east,the sun rises in the east,25,26,"['the', 'sun', 'rise', 'in', 'the', 'east']","['the', 'sun', 'rises', 'in', 'the', 'east']","[('the', 'DT'), ('sun', 'NN'), ('rise', 'NN'),...","[('the', 'DT'), ('sun', 'NN'), ('rises', 'VBZ'...",[ 0

In [27]:
# printing dataset columns
print(df.columns)

Index(['Error Type', 'Ungrammatical Statement', 'Standard English',
       'Ungrammatical Length', 'Standard Length', 'Tokenized Ungrammatical',
       'Tokenized Standard', 'POS_Ungrammatical', 'POS_Standard',
       'POS_W2V_Ungram', 'POS_W2V_Standard', 'POS_seq_U', 'POS_seq_S',
       'W2V_Ungram', 'W2V_Standard', 'Bigrams_Ungrammatical',
       'Bigrams_Standard', 'Trigrams_Ungrammatical', 'Trigrams_Standard',
       'W2V_Bigrams_Ungrammatical', 'W2V_Bigrams_Standard',
       'W2V_Trigrams_Ungrammatical', 'W2V_Trigrams_Standard'],
      dtype='object')


# Naive Bayes

In [30]:
# 1) Function to build a pipeline for one embedding pair
def make_embedding_pipeline(ug_col, st_col):
    """
    Returns a (pipeline, X_df, y) tuple ready to fit/cv:
      - X_df.feat holds the array features
      - y is the 0/1 label vector
      - pipeline stacks them and runs GaussianNB
    """
    n = len(df)
    # build X_df: first all ungrammatical, then all corrected arrays
    feats = list(df[ug_col]) + list(df[st_col])
    y     = np.array([0]*n + [1]*n)
    X_df  = pd.DataFrame({'feat': feats})

    # extractor: from X_df, pull 'feat' and vstack into a matrix
    extractor = FunctionTransformer(lambda X: np.vstack(X['feat']), validate=False)

    pipe = Pipeline([
        ('stack', extractor),
        ('clf',   GaussianNB())
    ])
    return pipe, X_df, y

# 2) Run it for each embedding type with a 90/10 holdout
for name, (ug, st) in embedding_experiments.items():
    pipe, X_df, y = make_embedding_pipeline(ug, st)

    # train/test split
    X_tr, X_te, y_tr, y_te = train_test_split(
        X_df, y,
        test_size=0.1,
        stratify=y,
        random_state=42
    )

    # fit & eval
    pipe.fit(X_tr, y_tr)
    preds = pipe.predict(X_te)
    acc   = np.mean(preds == y_te)
    print(f"{name} hold-out accuracy: {acc:.4f}")
    print(classification_report(y_te, preds, digits=4))

# 3) 10-fold CV
print("\n10-fold CV results:")
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for name, (ug, st) in embedding_experiments.items():
    pipe, X_df, y = make_embedding_pipeline(ug, st)
    scores = cross_val_score(pipe, X_df, y, cv=cv, scoring='accuracy')
    print(f"{name} CV accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

POS hold-out accuracy: 0.5149
              precision    recall  f1-score   support

           0     0.5197    0.3911    0.4463       202
           1     0.5119    0.6386    0.5683       202

    accuracy                         0.5149       404
   macro avg     0.5158    0.5149    0.5073       404
weighted avg     0.5158    0.5149    0.5073       404

Unigram hold-out accuracy: 0.5272
              precision    recall  f1-score   support

           0     0.5226    0.6287    0.5708       202
           1     0.5342    0.4257    0.4738       202

    accuracy                         0.5272       404
   macro avg     0.5284    0.5272    0.5223       404
weighted avg     0.5284    0.5272    0.5223       404

Bigram hold-out accuracy: 0.4975
              precision    recall  f1-score   support

           0     0.4982    0.6881    0.5780       202
           1     0.4960    0.3069    0.3792       202

    accuracy                         0.4975       404
   macro avg     0.4971    0.49